In [1]:
import numpy as np
import pandas as pd

import random
random.seed(0)
np.random.seed(0)

In [2]:
origin_data_path = '/home/ubuntu/data/PEC/dataset/ml-25m/ml-25m-sample.csv'

In [4]:
UIT_header = pd.read_csv(origin_data_path)
UIT_header

,u,i,time,day,video_size
0,441,15,2123,1,1
1,441,20,2123,1,1
2,441,9,2123,1,1
3,441,139,2123,1,1
4,441,312,2123,1,1
...,...,...,...,...,...
158018,880,6141,43140,29,1
158019,880,7973,43148,29,1
158020,880,9117,43148,29,1
158021,880,3669,43150,29,1


In [5]:
UIT_header = UIT_header.drop(['video_size'], axis=1)
UIT_header = UIT_header[['u', 'i', 'time']]
UIT_header.head()

,u,i,time
0,441,15,2123
1,441,20,2123
2,441,9,2123
3,441,139,2123
4,441,312,2123


In [7]:
len(set(UIT_header['u']))

1000

In [8]:
last_visited_videos = UIT_header.sort_values(by='time', ascending=False).groupby('u').first()
last_visited_videos.reset_index(inplace=True)

In [10]:
last_visited_videos

,u,i,time
0,0,702,33469
1,1,1713,32408
2,2,1837,7129
3,3,124,3312
4,4,187,2832
...,...,...,...
995,995,7784,29911
996,996,1700,32674
997,997,546,4205
998,998,369,2555


In [12]:
last_visited_videos.to_csv('Data_cRVR/origin_user.test.rating', sep='\t', lineterminator='\n', index=False,header=None)

In [14]:
for gamma in [0.01,0.025,0.05,0.075,0.1,0.25,0.5,0.75,1]:
    last_visited_videos.to_csv(f'Data/gamma_{gamma}.test.rating', sep='\t', lineterminator='\n', index=False,header=None)


In [15]:
import pandas as pd
import random

# 假设你有一个包含用户和视频索引的 DataFrame，命名为 df

# 获取所有唯一的用户和视频索引
unique_users = UIT_header['u'].unique()
unique_videos = UIT_header['i'].unique()

# 创建一个列表来存储用户没有访问过的视频索引
missing_videos = {}

# 遍历每个用户
for user in unique_users:
    # 找到该用户已经访问过的视频索引
    visited_videos = set(UIT_header[UIT_header['u'] == user]['i'])
    
    # 找到用户没有访问过的视频索引
    user_missing_videos = [video for video in unique_videos if video not in visited_videos]
    
    # 随机选择100个没有访问过的视频索引
    if len(user_missing_videos) >= 100:
        missing_videos[user] = random.sample(user_missing_videos, 100)
    else:
        missing_videos[user] = user_missing_videos

In [16]:
# 创建一个新的 DataFrame，包含用户索引和访问的视频索引
testing_notwatch_records = []

for index, row in last_visited_videos.iterrows():
    user = row['u'] 
    video = row['i']
    x = [(user,video)]
    for not_video in missing_videos[user]:
        x.append(not_video)
    testing_notwatch_records.append(x)

# 将训练记录转换为 DataFrame
testing_notwatch_records = pd.DataFrame(testing_notwatch_records)
# testing_notwatch_records[1] = testing_notwatch_records[1].apply(lambda x: ','.join(map(str, x)))
# testing_notwatch_records[1] = testing_notwatch_records[1].str.split(',')
# testing_notwatch_records = testing_notwatch_records.explode(1, ignore_index=True)

testing_notwatch_records.shape

(1000, 101)

In [17]:
testing_notwatch_records.to_csv('Data_cRVR/origin_user.test.negative', sep='\t', lineterminator='\n', index=False,header=None)

In [18]:
for gamma in [0.01,0.025,0.05,0.075,0.1,0.25,0.5,0.75,1.0]:
    testing_notwatch_records.to_csv(f'Data_cRVR/gamma_{gamma}.test.negative', sep='\t', lineterminator='\n', index=False,header=None)


In [19]:
import pandas as pd
import random

# 获取所有唯一的用户和视频索引
unique_users = UIT_header['u'].unique()
unique_videos = UIT_header['i'].unique()

# 创建一个列表来存储用户没有访问过的视频索引
wacth_videos = []

# 遍历每个用户
for user in unique_users:
    # 找到该用户已经访问过的视频索引
    visited_videos = set(UIT_header[UIT_header['u'] == user]['i'])
    for video in visited_videos:
        wacth_videos.append([user, video, 1.0])
wacth_videos = pd.DataFrame(wacth_videos)
print(wacth_videos.shape)
wacth_videos.to_csv('Data_cRVR/origin_user.train.rating', sep='\t', lineterminator='\n', index=False,header=None)

(158023, 3)


In [29]:
len(unique_videos)

10002

In [44]:
for gamma in [0.01,0.025,0.05,0.075,0.1,0.25,0.5,0.75,1.0]:
    action_path = f"/home/ubuntu/data/PEC/result_r_w/change_gamma/r_changeU_ecn-PEC_eds-PEC_ecn_beta_0.1_eds_gamma_{gamma}_eds_beta_0.1_a_upround_100.csv"
    #/home/ubuntu/data/PEC/result_r_w/change_gamma/r_changeU_ecn-PEC_eds-PEC_ecn_beta_0.1_eds_gamma_0.01_eds_beta_0.1_a_upround_100_01-12-22-43-13.csv
    redundant_action = np.loadtxt(action_path, delimiter=",")
    users_request_videos = []
    # 遍历每个用户
    for user in range(redundant_action.shape[0]):
        request_videos = set(np.argwhere(redundant_action[user]>0)[:,0])
        for video in request_videos:
            users_request_videos.append([user, video, 1.0])
    pd.DataFrame(users_request_videos).to_csv(f'Data_cRVR/gamma_{gamma}.train.rating', sep='\t', lineterminator='\n', index=False,header=None)

In [41]:
users_request_videos = []
# 遍历每个用户
for user in range(redundant_action.shape[0]):
    request_videos = set(np.argwhere(redundant_action[user]>0)[:,0])
    for video in request_videos:
        users_request_videos.append([user, video, 1.0])

gamma = 0.1
pd.DataFrame(users_request_videos).to_csv(f'Data_cRVR/gamma_{gamma}.train.rating', sep='\t', lineterminator='\n', index=False,header=None)

In [13]:
action_path = "/home/ubuntu/data/PPF/final_state/final_actions/use/fe/09-27-14-22-15-GREED-HPP_PAC-testing-Noise:True-ce:0.01-xi:10.0-epsilon:1.0-delta:0.1-dataset:1.0.npy"
redundant_actions_greed =np.load(action_path)

action_path = "/home/ubuntu/data/PPF/final_state/final_actions/use/fe/09-27-14-22-09-INFOCOM-HPP_PAC-testing-Noise:True-ce:0.01-xi:10.0-epsilon:1.0-delta:0.1-dataset:1.0.npy"

redundant_actions_infocom =np.load(action_path)

action_path = "/home/ubuntu/data/PPF/final_state/final_actions/use/fe/09-27-14-22-22-RANDOM-HPP_PAC-testing-Noise:True-ce:0.01-xi:10.0-epsilon:1.0-delta:0.1-dataset:1.0.npy"

redundant_actions_random =np.load(action_path)

random.seed(0)

cache_actions_greed = redundant_actions_greed[:,0,:]
cache_actions_random = redundant_actions_random[:,0,:]
cache_actions_infocom = redundant_actions_infocom[:,0,:]

assert (cache_actions_greed==cache_actions_greed).all()
wacth_videos_greed = []
wacth_videos_random = []
wacth_videos_infocom = []
ed_profile_greed = np.zeros((ED_num,content_num),dtype=np.int64)
ed_profile_random = np.zeros((ED_num,content_num),dtype=np.int64)
ed_profile_infocom = np.zeros((ED_num,content_num),dtype=np.int64)

for ed_index in all_ED_index:
    ed_profile_greed[ed_index] = np.where(cache_actions_greed[ed_index] > 0, 1, 0)
    ed_profile_random[ed_index] = np.where(cache_actions_random[ed_index] > 0, 1, 0)
    ed_profile_infocom[ed_index] = np.where(cache_actions_infocom[ed_index] > 0, 1, 0)

    assert (ed_profile_greed[ed_index]==ed_profile_random[ed_index]).all()
    request_video_greed = set(np.argwhere(ed_profile_greed[ed_index]>0)[:,0])
    request_video_random = set(np.argwhere(ed_profile_random[ed_index]>0)[:,0])
    request_video_infocom = set(np.argwhere(ed_profile_infocom[ed_index]>0)[:,0])

    assert request_video_greed==request_video_random and request_video_greed==request_video_random
    for video in request_video_random:
        # if video in ed_to_video_sample[ed_index]:
        # user = random.choice(list(ed_to_user_sample[ed_index]))
            # for user in ed_to_user_sample[ed_index]:
        user = random.choice(ed_to_user[ed_index])
        wacth_videos_infocom.append([user, video, 1.0])
        wacth_videos_random.append([user, video, 1.0])
        wacth_videos_greed.append([user, video, 1.0])

    assert (np.array(wacth_videos_greed)==np.array(wacth_videos_random)).all() and (np.array(wacth_videos_greed)==np.array(wacth_videos_infocom)).all()
pd.DataFrame(wacth_videos_random).to_csv(f'Data/fe_greed_{0}.train.rating', sep='\t', lineterminator='\n', index=False,header=None)
pd.DataFrame(wacth_videos_infocom).to_csv(f'Data/fe_infocom_{0}.train.rating', sep='\t', lineterminator='\n', index=False,header=None)
pd.DataFrame(wacth_videos_greed).to_csv(f'Data/fe_random_{0}.train.rating', sep='\t', lineterminator='\n', index=False,header=None)

In [17]:
# infocom fe:[1, 2, 3, 4, 5, 6]
action_path = '/home/ubuntu/data/PPF/final_state/final_actions/use/fe/09-27-14-22-15-GREED-HPP_PAC-testing-Noise:True-ce:0.01-xi:10.0-epsilon:1.0-delta:0.1-dataset:1.0.npy'


redundant_actions =np.load(action_path)

for cache_index in range(1,redundant_actions.shape[1]):
    cache_actions = redundant_actions[:,cache_index,:]
    wacth_videos = []
    ed_profile = np.zeros((ED_num,content_num),dtype=np.int64)
    for ed_index in all_ED_index:
        ed_profile[ed_index] = np.where(cache_actions[ed_index] > 0, 1, 0)
        request_video = set(np.argwhere(ed_profile[ed_index]>0)[:,0])
        for video in request_video:
            # if video in ed_to_video_sample[ed_index]:
            # user = random.choice(list(ed_to_user_sample[ed_index]))
            user = random.choice(ed_to_user[ed_index])
            wacth_videos.append([user, video, 1.0])
    print(cache_index,len(wacth_videos))
    pd.DataFrame(wacth_videos).to_csv(f'Data/fe_infocom_{cache_index}.train.rating', sep='\t', lineterminator='\n', index=False,header=None)


1 119097
2 139883
3 169532
4 204643
5 243597
6 249861


In [18]:
# random fe:[1, 2, 3, 4, 5, 6]
action_path ='/home/ubuntu/data/PPF/final_state/final_actions/use/fe/09-27-14-22-09-INFOCOM-HPP_PAC-testing-Noise:True-ce:0.01-xi:10.0-epsilon:1.0-delta:0.1-dataset:1.0.npy'


redundant_actions =np.load(action_path)

for cache_index in range(1,redundant_actions.shape[1]):
    cache_actions = redundant_actions[:,cache_index,:]
    wacth_videos = []
    ed_profile = np.zeros((ED_num,content_num),dtype=np.int64)
    for ed_index in all_ED_index:
        ed_profile[ed_index] = np.where(cache_actions[ed_index] > 0, 1, 0)
        request_video = set(np.argwhere(ed_profile[ed_index]>0)[:,0])
        for video in request_video:
            # if video in ed_to_video_sample[ed_index]:
            # user = random.choice(list(ed_to_user_sample[ed_index]))
                # for user in ed_to_user_sample[ed_index]:
            user = random.choice(ed_to_user[ed_index])
            wacth_videos.append([user, video, 1.0])
    print(cache_index,len(wacth_videos))
    pd.DataFrame(wacth_videos).to_csv(f'Data/fe_random_{cache_index}.train.rating', sep='\t', lineterminator='\n', index=False,header=None)


1 164374
2 217984
3 245137
4 245851
5 246302
6 246358


In [19]:
# greed ce
action_path = '/home/ubuntu/data/PPF/final_state/final_actions/use/fe/09-27-14-22-22-RANDOM-HPP_PAC-testing-Noise:True-ce:0.01-xi:10.0-epsilon:1.0-delta:0.1-dataset:1.0.npy'


redundant_actions =np.load(action_path)

for cache_index in range(1,redundant_actions.shape[1]):
    cache_actions = redundant_actions[:,cache_index,:]
    wacth_videos = []
    ed_profile = np.zeros((ED_num,content_num),dtype=np.int64)
    for ed_index in all_ED_index:
        ed_profile[ed_index] = np.where(cache_actions[ed_index] > 0, 1, 0)
        request_video = set(np.argwhere(ed_profile[ed_index]>0)[:,0])
        for video in request_video:
            # if video in ed_to_video_sample[ed_index]:
            # user = random.choice(list(ed_to_user_sample[ed_index]))
            user = random.choice(ed_to_user[ed_index])
                # for user in ed_to_user_sample[ed_index]:
            wacth_videos.append([user, video, 1.0])
    print(cache_index,len(wacth_videos))
    pd.DataFrame(wacth_videos).to_csv(f'Data/fe_greed_{cache_index}.train.rating', sep='\t', lineterminator='\n', index=False,header=None)


1 139121
2 166272
3 193398
4 220699
5 246056
6 249795


In [17]:
# fifo 
action_path = "/home/ubuntu/data/PPF/final_state/final_actions/use/fe/09-30-12-50-42-FIFO-NO_MODEL-testing-Noise:False-ce:[0.001, 0.0025, 0.005, 0.01, 0.025, 0.05, 0.1]-delta:0.1-dataset:1.0.npy"

random.seed(0)
redundant_actions =np.load(action_path)

cache_actions = redundant_actions[:,3,:]
fifo_wacth_videos = []
ed_profile = np.zeros((ED_num,content_num),dtype=np.int64)
for ed_index in all_ED_index:
    ed_profile[ed_index] = np.where(cache_actions[ed_index] > 0, 1, 0)
    request_video = set(np.argwhere(ed_profile[ed_index]>0)[:,0])
    for video in request_video:
        # if video in ed_to_video_sample[ed_index]:
        # user = random.choice(list(ed_to_user_sample[ed_index]))
        user = random.choice(ed_to_user[ed_index])
            # for user in ed_to_user_sample[ed_index]:
        fifo_wacth_videos.append([user, video, 1.0])
pd.DataFrame(fifo_wacth_videos).to_csv(f'Data/fifo.train.rating', sep='\t', lineterminator='\n', index=False,header=None)


In [18]:
# lru
lru_action_path = "/home/ubuntu/data/PPF/final_state/final_actions/use/fe/09-30-12-50-50-LRU-NO_MODEL-testing-Noise:False-ce:[0.001, 0.0025, 0.005, 0.01, 0.025, 0.05, 0.1]-delta:0.1-dataset:1.0.npy"

random.seed(0)
redundant_actions =np.load(action_path)

cache_actions = redundant_actions[:,3,:]
lru_wacth_videos = []
ed_profile = np.zeros((ED_num,content_num),dtype=np.int64)
for ed_index in all_ED_index:
    ed_profile[ed_index] = np.where(cache_actions[ed_index] > 0, 1, 0)
    request_video = set(np.argwhere(ed_profile[ed_index]>0)[:,0])
    for video in request_video:
        # if video in ed_to_video_sample[ed_index]:
        # user = random.choice(list(ed_to_user_sample[ed_index]))
        user = random.choice(ed_to_user[ed_index])
            # for user in ed_to_user_sample[ed_index]:
        lru_wacth_videos.append([user, video, 1.0])
    pd.DataFrame(fifo_wacth_videos).to_csv(f'Data/lru.train.rating', sep='\t', lineterminator='\n', index=False,header=None)


In [19]:
# lfu
lru_action_path = "/home/ubuntu/data/PPF/final_state/final_actions/use/fe/09-30-12-50-57-LFU-NO_MODEL-testing-Noise:False-ce:[0.001, 0.0025, 0.005, 0.01, 0.025, 0.05, 0.1]-delta:0.1-dataset:1.0.npy"

random.seed(0)
redundant_actions =np.load(action_path)

cache_actions = redundant_actions[:,3,:]
lru_wacth_videos = []
ed_profile = np.zeros((ED_num,content_num),dtype=np.int64)
for ed_index in all_ED_index:
    ed_profile[ed_index] = np.where(cache_actions[ed_index] > 0, 1, 0)
    request_video = set(np.argwhere(ed_profile[ed_index]>0)[:,0])
    for video in request_video:
        # if video in ed_to_video_sample[ed_index]:
        # user = random.choice(list(ed_to_user_sample[ed_index]))
        user = random.choice(ed_to_user[ed_index])
            # for user in ed_to_user_sample[ed_index]:
        lru_wacth_videos.append([user, video, 1.0])
    pd.DataFrame(fifo_wacth_videos).to_csv(f'Data/lru.train.rating', sep='\t', lineterminator='\n', index=False,header=None)


In [11]:
# infocom xi:[5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0]
action_path = "/home/ubuntu/data/PPF/final_state/final_actions/10-01-15-22-08-INFOCOM-HPP_PAC-testing-Noise:True-fe:3-ce:0.01-xi:[5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0]-epsilon:1.0-dataset:1.0.npy"

redundant_actions =np.load(action_path)

for cache_index in range(redundant_actions.shape[1]):
    cache_actions = redundant_actions[:,cache_index,:]
    wacth_videos = []
    ed_profile = np.zeros((ED_num,content_num),dtype=np.int64)
    for ed_index in all_ED_index:
        ed_profile[ed_index] = np.where(cache_actions[ed_index] > 0, 1, 0)
        request_video = set(np.argwhere(ed_profile[ed_index]>0)[:,0])
        for video in request_video:
            # if video in ed_to_video_sample[ed_index]:
            # user = random.choice(list(ed_to_user_sample[ed_index]))
            user = random.choice(ed_to_user[ed_index])
            wacth_videos.append([user, video, 1.0])
    print(cache_index,len(wacth_videos))
    pd.DataFrame(wacth_videos).to_csv(f'Data/xi_infocom_{cache_index}.train.rating', sep='\t', lineterminator='\n', index=False,header=None)


0 230835
1 240881
2 245185
3 244323
4 226458
5 212078
6 200909


In [12]:
# GREED xi:[5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0]
action_path = "/home/ubuntu/data/PPF/final_state/final_actions/10-01-15-22-00-GREED-HPP_PAC-testing-Noise:True-fe:3-ce:0.01-xi:[5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0]-epsilon:1.0-dataset:1.0.npy"

redundant_actions =np.load(action_path)

for cache_index in range(0,redundant_actions.shape[1]):
    cache_actions = redundant_actions[:,cache_index,:]
    wacth_videos = []
    ed_profile = np.zeros((ED_num,content_num),dtype=np.int64)
    for ed_index in all_ED_index:
        ed_profile[ed_index] = np.where(cache_actions[ed_index] > 0, 1, 0)
        request_video = set(np.argwhere(ed_profile[ed_index]>0)[:,0])
        for video in request_video:
            # if video in ed_to_video_sample[ed_index]:
            # user = random.choice(list(ed_to_user_sample[ed_index]))
                # for user in ed_to_user_sample[ed_index]:
            user = random.choice(ed_to_user[ed_index])
            wacth_videos.append([user, video, 1.0])
    print(cache_index,len(wacth_videos))
    pd.DataFrame(wacth_videos).to_csv(f'Data/xi_greed_{cache_index}.train.rating', sep='\t', lineterminator='\n', index=False,header=None)


0 247668
1 196816
2 159824
3 147112
4 134781
5 130474
6 126209


In [13]:
# random  xi:[5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0]
action_path = '/home/ubuntu/data/PPF/final_state/final_actions/10-01-15-21-54-RANDOM-HPP_PAC-testing-Noise:True-fe:3-ce:0.01-xi:[5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0]-epsilon:1.0-dataset:1.0.npy'

redundant_actions =np.load(action_path)

for cache_index in range(0,redundant_actions.shape[1]):
    cache_actions = redundant_actions[:,cache_index,:]
    wacth_videos = []
    ed_profile = np.zeros((ED_num,content_num),dtype=np.int64)
    for ed_index in all_ED_index:
        ed_profile[ed_index] = np.where(cache_actions[ed_index] > 0, 1, 0)
        request_video = set(np.argwhere(ed_profile[ed_index]>0)[:,0])
        for video in request_video:
            # if video in ed_to_video_sample[ed_index]:
            # user = random.choice(list(ed_to_user_sample[ed_index]))
            user = random.choice(ed_to_user[ed_index])
                # for user in ed_to_user_sample[ed_index]:
            wacth_videos.append([user, video, 1.0])
    print(cache_index,len(wacth_videos))
    pd.DataFrame(wacth_videos).to_csv(f'Data/xi_random_{cache_index}.train.rating', sep='\t', lineterminator='\n', index=False,header=None)


0 246951
1 214906
2 184752
3 173268
4 161729
5 156106
6 149928
